One is commonly interested in using different loss functions. Luckily for us, LightGBM allows us to plug in basically any loss we want.

To do that, we also need to provide:

* The Jacobian with respect to the model parameters
* The Hessian.

In [1]:
import warnings 

from typing import AnyStr, Callable

import numpy as np
import lightgbm as lgb
import sympy as sym

from scipy.special import expit

In [2]:
# setup
y = sym.symbols('y', real=True, constant=True)
p = sym.symbols('p', real=True, positive=True)

# loss
loss = - y * sym.log(p) - (1-y) * sym.log(1-p)
loss

-y*log(p) - (1 - y)*log(1 - p)

In [3]:
# recall derivative of p, which is a logit
dp = p*(1-p)

# take derivatives (*dp to manually add chain rule)
jacob = sym.simplify(sym.diff(loss, p) * dp) 
hessian = sym.simplify(sym.diff(jacob, p) * dp)  

In [9]:
jacob

p - y

In [10]:
hessian

p*(1 - p)

In [4]:
params = (y,p)
loss_func    = sym.lambdify(params, loss)
jacob_func   = sym.lambdify(params, jacob)
hessian_func = sym.lambdify(params, hessian)

In [5]:
class CustomLoss:

    def __init__(self, 
                 loss_func: Callable,
                 jacob_func: Callable,
                 hessian_func: Callable,
                 loss_name="custom_loss"):
        
        self.loss = loss_func
        self.jacob = jacob_func
        self.hessian = hessian_func

    def p(self, y_probs):
        return np.clip(y_probs, 1e-15, 1)
        
    def __call__(self, y_true, y_probs):
        p = self.p(y_probs)
        return self.loss(y=y_true, p=p) #.mean()

    def grad(self, y_true, y_probs):
        p = self.p(y_probs)
        return self.jacob(y=y_true, p=p)

    def hess(self, y_true, y_probs):
        p = self.p(y_probs)
        return self.hessian(y=y_true, p=p)

    def init_score(self, y_true):
        from scipy import optimize

        res = optimize.minimize_scalar(
            lambda p: self(y_true, p).sum(),
            bounds=(0, 1),
            method='bounded'
        )
        p = res.x
        log_odds = np.log(p / (1 - p))
        return log_odds

    def lgb_obj(self, preds, train_data):
        y = train_data.get_label()
        p = expit(preds)
        return self.grad(y, p), self.hess(y, p)

    def lgb_eval(self, preds, train_data):
        y = train_data.get_label()
        p = expit(preds)
        is_higher_better = False
        return 'focal_loss', self(y, p).mean(), is_higher_better

    
class CustomLossLGBM(lgb.LGBMClassifier):

    def __init__(self, custom_loss: CustomLoss, **kwargs):
        self.params = kwargs
        self.fl = custom_loss
        self._other_params = []
    
    def _fit_optimal_rounds(self, fit_data, max_boost_round, early_stopping_rounds):
        "use this with early_stopping to find optimal number of rounds"

        classifier = lgb.Booster(
            params=self.params, 
            train_set=fit_data,
        )

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            results = lgb.cv(
                init_model=classifier,
                params=self.params, 
                train_set=fit_data,
                nfold=2,
                num_boost_round=max_boost_round,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=False,
                fobj=self.fl.lgb_obj,
                feval=self.fl.lgb_eval
            )
        
        return len(results['focal_loss-mean'])

    def fit(self, X_fit, y_fit, max_boost_round=1000, early_stopping_rounds=20):
        
        self.init_score = self.fl.init_score(y_fit)

        fit_data = lgb.Dataset(
            X_fit, y_fit,
            init_score=np.full_like(y_fit, self.init_score, dtype=float),
            free_raw_data=False
        )
        
        self.optimal_boosting_rounds = self._fit_optimal_rounds(fit_data,
                                                                max_boost_round, early_stopping_rounds)
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            model = lgb.train(
                params=self.params,
                num_boost_round=self.optimal_boosting_rounds,
                train_set=fit_data,
                fobj=self.fl.lgb_obj,
                feval=self.fl.lgb_eval
            )
        
        self.model = model
        return self
        
    def predict_proba(self, X):
        prob_1 =  expit(self.init_score + self.model.predict(X))
        prob_0 = 1 - prob_1
        return np.array([prob_0, prob_1]).T

    def predict(self, X):
        pass

In [6]:
# Testing and benchmarking

from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier


def make_balanced_problem(n_samples=5000):
    X, y = make_classification(n_samples=n_samples, n_features=10, n_informative=8, n_redundant=1, n_repeated=1, 
                               random_state=10) 
    return X, y

X, y = make_balanced_problem(n_samples=15000)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [7]:
model = CustomLossLGBM(custom_loss=CustomLoss(loss_func, jacob_func, hessian_func),
                      learning_rate= 0.1,
                      n_estimators=500,
                      num_leaves=63,
                      n_jobs=10,
                      verbose=-1, random_state=0)

In [8]:
model.fit(X_train, y_train)
y_probs = model.predict_proba(X_test)[:,1]

print('  AUC  : {0:.3f}'.format(roc_auc_score(y_test, y_probs)))
print('  AP   : {0:.3f}'.format(average_precision_score(y_test, y_probs)))
print('  Brier: {0:.3f}'.format(brier_score_loss(y_test, y_probs)))

  AUC  : 0.986
  AP   : 0.984
  Brier: 0.037
